In [2]:
# Install gdown jika belum ada
!pip install gdown

# File ID dan nama output
file_id = '1sfWNpOGVHZ8AXjTJ0mS-ryZousfFFB6R'
output_file = 'Dataset.zip'

# Generate link download
download_link = f'https://drive.google.com/uc?id={file_id}'

# Download file
!gdown $download_link -O $output_file

Downloading...
From (original): https://drive.google.com/uc?id=1sfWNpOGVHZ8AXjTJ0mS-ryZousfFFB6R
From (redirected): https://drive.google.com/uc?id=1sfWNpOGVHZ8AXjTJ0mS-ryZousfFFB6R&confirm=t&uuid=6a5381d1-6c2f-4e1b-b5fd-2d3341119513
To: /kaggle/working/Dataset.zip
100%|████████████████████████████████████████| 432M/432M [00:05<00:00, 84.9MB/s]


In [3]:
!unzip /kaggle/working/Dataset.zip

Archive:  /kaggle/working/Dataset.zip
   creating: dataset-season/
   creating: dataset-season/season/
   creating: dataset-season/season/test/
   creating: dataset-season/season/test/dry/
  inflating: dataset-season/season/test/dry/Salinan 01(1).jpg  
  inflating: dataset-season/season/test/dry/Salinan 01(2).jpg  
  inflating: dataset-season/season/test/dry/Salinan 01.jpg  
  inflating: dataset-season/season/test/dry/Salinan 02(1).jpg  
  inflating: dataset-season/season/test/dry/Salinan 02.jpg  
  inflating: dataset-season/season/test/dry/Salinan 03(1).jpg  
  inflating: dataset-season/season/test/dry/Salinan 03.jpg  
  inflating: dataset-season/season/test/dry/Salinan 04(1).jpg  
  inflating: dataset-season/season/test/dry/Salinan 04.jpg  
  inflating: dataset-season/season/test/dry/Salinan 05(1).jpg  
  inflating: dataset-season/season/test/dry/Salinan 05.jpg  
  inflating: dataset-season/season/test/dry/Salinan 06(1).jpg  
  inflating: dataset-season/season/test/dry/Salinan 06.jpg

In [4]:
import os

import numpy as np

import tensorflow as tf

import tensorflow_hub as hub

import matplotlib.pyplot as plt

from tensorflow.keras.utils import load_img, img_to_array

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
train_dir = "/kaggle/working/dataset-season/season/train"

test_dir = "/kaggle/working/dataset-season/season/test"

validation_dir = "/kaggle/working/dataset-season/season/valid"



sorted_labels = [
    'dry', 'rainy'
]

# train directories
train_directories = {label: os.path.join(train_dir, label) for label in sorted_labels}

test_directories = {label: os.path.join(test_dir, label) for label in sorted_labels}

# validation directories
validation_directories = {label: os.path.join(validation_dir, label) for label in sorted_labels}

for label in sorted_labels:
    print('Total training', label, 'images:', len(os.listdir(train_directories[label])))

for label in sorted_labels:
    print('Total test', label, 'images:', len(os.listdir(test_directories[label])))

for label in sorted_labels:
    print('Total validation', label, 'images:', len(os.listdir(validation_directories[label])))

blazers = os.listdir(train_directories['dry'])
print(blazers[:5])

Total training dry images: 735
Total training rainy images: 840
Total test dry images: 158
Total test rainy images: 180
Total validation dry images: 157
Total validation rainy images: 180
['Salinan 47(9).jpg', 'Salinan 06(10).jpg', 'Salinan 48(8).jpg', 'Salinan 19(19).jpg', 'Salinan 01(7).jpg']


In [9]:
# Feature extractor layer

feature_extractor_url = 'https://tfhub.dev/google/efficientnet/b7/feature-vector/1'

feature_extractor_layer = hub.KerasLayer(feature_extractor_url, input_shape=[512,512,3])

feature_extractor_layer.trainable = False

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(224, 224, 3)),
    tf.keras.layers.Lambda(lambda x: feature_extractor_layer(x)),
    tf.keras.layers.Dense(512, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.001)),  # Regularisasi L2
    tf.keras.layers.Dropout(0.5),  # Tingkatkan dropout rate
    tf.keras.layers.Dense(256, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.001)),  # Regularisasi L2
    tf.keras.layers.Dropout(0.3),  # Tambahkan dropout lagi
    tf.keras.layers.Dense(2, activation="softmax") 
])

model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lambda (Lambda)                 │ (None, 2560)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,311,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,443,074 (5.50 MB)

 Trainable params: 1,443,074 (5.50 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.compile(optimizer= 'Adam',

              loss= 'categorical_crossentropy',

              metrics= ['accuracy'])

In [12]:
# Add data-augmentation parameters to ImageDataGenerator
from PIL import Image

def remove_transparency(image):
    if image.mode in ('RGBA', 'LA') or (image.mode == 'P' and 'transparency' in image.info):
        background = Image.new('RGB', image.size, (255, 255, 255))
        background.paste(image, mask=image.split()[3] if 'transparency' in image.info else None)
        return background
    else:
        return image


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,  # Tingkatkan rentang rotasi
    width_shift_range=0.3,  # Tingkatkan rentang pergeseran horizontal
    height_shift_range=0.3,  # Tingkatkan rentang pergeseran vertikal
    shear_range=0.3,  # Tingkatkan rentang shear
    zoom_range=0.3,  # Tingkatkan rentang zoom
    horizontal_flip=True, 
    brightness_range=[0.7, 1.3],  # Rentang kecerahan lebih lebar
    fill_mode='nearest',
)

# Flow training images in batches of 32 using train_datagen generator

train_generator = train_datagen.flow_from_directory(train_dir,

                                                    batch_size= 32,

                                                    class_mode= 'categorical',

                                                    target_size= (224, 224))



validation_datagen = ImageDataGenerator(rescale = 1./255)



# Flow validation images in batches of 32 using validation_datagen generator

validation_generator = validation_datagen.flow_from_directory(validation_dir,

                                                    batch_size= 32,

                                                    class_mode= 'categorical',

                                                    target_size= (224, 224))

Found 1575 images belonging to 2 classes.
Found 337 images belonging to 2 classes.


In [ ]:
class Callback(tf.keras.callbacks.Callback):

  def on_epoch_end(self, epoch, logs={}):

    if(logs.get('accuracy')>0.9000):

      print("\nReached 90% accuracy so cancelling training!")

      self.model.stop_training = True



callbacks = Callback()



# Train the model

history = model.fit(train_generator,

                    epochs= 35,

                    validation_data= validation_generator,

                    verbose= 1,

                    callbacks= callbacks)

Epoch 1/35


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


13/50 ━━━━━━━━━━━━━━━━━━━━ 6:33 11s/step - accuracy: 0.4921 - loss: 1.9183

/opt/conda/lib/python3.10/site-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


50/50 ━━━━━━━━━━━━━━━━━━━━ 658s 13s/step - accuracy: 0.5785 - loss: 1.8228 - val_accuracy: 0.4688 - val_loss: 1.8639
Epoch 2/35
50/50 ━━━━━━━━━━━━━━━━━━━━ 640s 13s/step - accuracy: 0.6828 - loss: 1.5263 - val_accuracy: 0.5134 - val_loss: 1.6187
Epoch 3/35
 8/50 ━━━━━━━━━━━━━━━━━━━━ 7:40 11s/step - accuracy: 0.6579 - loss: 1.3887

In [ ]:
# Plot the results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

# Plot the accuracy
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

# Plot the loss
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
# Predictions from validation folder
from tensorflow.keras.preprocessing import image

for label in sorted_labels:
  folder = validation_directories[label]

  for path_file in os.listdir(folder):
    # Get value from image
    path_file = os.path.join(folder, path_file)
    img = image.load_img(path_file, target_size=(512, 512))
    x = img_to_array(img)
    x = x/255
    x = np.expand_dims(x, axis=0)

    # Predict the image label
    images = np.vstack([x])
    classes = model.predict(images, batch_size=31)

    # Take the class index with the highest probability
    predicted_class_index = classes.argmax()

    # Convert class indices to label
    predicted_label = sorted_labels[predicted_class_index]

    print("Label:", label, "Predicted Label:", predicted_label)

In [ ]:
model.save("model-season.h5")

In [ ]:
import shutil

# Pindahkan file ke lokasi tertentu
shutil.move('model-season.h5', '/kaggle/working/model-season.h5')

In [1]:
model.download()

NameError: name 'model' is not defined